Importing all the libraries

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

Making a beautifulsoup object given the site link.

In [2]:
URL = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

Finding the table in the html code.

In [3]:
table=soup.find('table')

Creating an inital rough table with all the details.

In [4]:
table_contents=[]
for row in table.findAll('tr')[1:]:
    cell = {}
    if row.findAll('td')[1:2][0].text=='Not assigned':     ## If 'Borough' is 'Not assigned' then we simply ignore the row
        pass
    else:
        cell['PostalCode'] = row.td.text[:3]
        cell['Borough'] = row.findAll('td')[1:2][0].text.strip("\n")
        
        ## If 'Neighborhood' is 'Not assigned' then we give it the value from 'Borough'
        if row.findAll('td')[2:3][0].text == 'Not assigned\n':
            cell['Neighborhood'] = row.findAll('td')[1:2][0].text.strip("\n")
        else:
            cell['Neighborhood'] = row.findAll('td')[2:3][0].text.strip("\n")
        
        ## Appending it to the list
        table_contents.append(cell)

Converting list to dataframes.

In [5]:
df=pd.DataFrame(table_contents)

Combining cells with same 'Postal Code'.

In [6]:
df['Neighborhood'] = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].transform(lambda x: ' '.join(x))
df = df.drop_duplicates()
print(df.head(10))

   PostalCode           Borough                     Neighborhood
0         M3A        North York                        Parkwoods
1         M4A        North York                 Victoria Village
2         M5A  Downtown Toronto         Harbourfront Regent Park
4         M6A        North York  Lawrence Heights Lawrence Manor
6         M7A      Queen's Park                     Queen's Park
7         M9A         Etobicoke                 Islington Avenue
8         M1B       Scarborough                    Rouge Malvern
10        M3B        North York                  Don Mills North
11        M4B         East York   Woodbine Gardens Parkview Hill
13        M5B  Downtown Toronto          Ryerson Garden District


Printing the shape.

In [7]:
print(df.shape)

(103, 3)
